In [21]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
from keras import backend as K
import os
import keras
import tensorflow as tf
import random as rn
from keras.utils.generic_utils import get_custom_objects




In [17]:
#reproducibility

os.environ['PYTHONHASHSEED'] ='0'
#Below is necessary for starting Numpy generated 
#random numbers in a well-defined initial state
np.random.seed(42)

#Below is necessary for starting core Python generated 
#random numbers in a well-defined initial state
rn.seed(12345)
'''force tensorflow to use single thread
   multiple threads are the potential source
   of non-reproducible results
'''
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#The below tf.set_random_seed() will make random number generation in
#Tensorflow backend have well defined initial state
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [18]:
#Define activation functions
def tf_sqnlsig(x):
    u=tf.clip_by_value(x,-2,2)
    a = u
    b= tf.negative(tf.abs(u))
    wsq = (tf.multiply(a,b))/4.0
    y = tf.add(tf.multiply(tf.add(u,wsq),0.5),0.5)
    return y
get_custom_objects().update({'custom_activation': Activation(tf_sqnlsig)})
def tf_sqnl(x): 
    u=tf.clip_by_value(x,-2,2)
    a = u
    b= tf.negative(tf.abs(u))
    wsq = (tf.multiply(a,b))/4.0
    y = tf.add(u,wsq)
    return y
get_custom_objects().update({'custom_activation': Activation(tf_sqnl)})

def tf_tansig(x): 
    w=tf.multiply(tf.negative(2.0), x)
    u=tf.exp(w)
    a = 1.0 + u
    b= 2.0/a
    y = b - 1.0
    return y
get_custom_objects().update({'custom_activation': Activation(tf_tansig)})

def tf_logsig(x): 
    u=tf.exp(tf.negative(x)) 
    a = tf.add(u,1.0)
    y= 1.0/a
    return y
get_custom_objects().update({'custom_activation': Activation(tf_logsig)})

In [19]:
#Get Data and preprocess
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...


In [22]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
#keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid'#keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid'
#model.add(LSTM(128, input_shape=(maxlen, len(chars))))
#model.add(keras.layers.LSTM(128 , input_shape=(maxlen, len(chars)), activation=tf_tansig, recurrent_activation=tf_logsig))
model.add(keras.layers.LSTM(128 , input_shape=(maxlen, len(chars)),  activation=tf_sqnl, recurrent_activation=tf_sqnlsig))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [23]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()



In [24]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=4,
          callbacks=[print_callback])

Epoch 1/4
200285/200285 [==============================] - 189s 944us/step - loss: 2.1537

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "uxiliaries; nevertheless, the sum of
the"
uxiliaries; nevertheless, the sum of
the person and the and the meral the proman and the greation and the person and the man and the because of the the sentice, and the something and the will the proble and the perman and the sentice and the ham the person and all the stilly in the mantient and and the which the sectance and the mandices and the sentices and the manist and in the mandices and the mentices and and the senting and the man
----- diversity: 0.5
----- Generating with seed: "uxiliaries; nevertheless, the sum of
the"
uxiliaries; nevertheless, the sum of
the man of the prowes the lake and the juded as is the manity the geadine of the scient and incontices and and concempation in the man
and man and surtame
and to the contial spiritual is entimes the morality an

/Users/adedamola/Tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 all the say and a survicated and in the say and the supersatisment the for the power of the of the sighted and the man and something the something and the form and the for the all the say and a surv
----- diversity: 0.5
----- Generating with seed: " long to visit at the most
decisive time"
 long to visit at the most
decisive times of the formority in the possesseries of the in the bey whole man in the life in the immoraley the
searned and spirit we have there is a sense? in the in, that the say in the in the in the inforestly, in the power to the
cannal and man
and desirefoly a sour musing word
sensive of the wordinal the are have
-the account of the approne in the all loves they themoration for the happiness the somethin
----- diversity: 1.0
----- Generating with seed: " long to visit at the most
decisive time"
 long to visit at the most
decisive times anothes theishow dangelish doinons, a griech, and whantimative moral of hart, to at in men, the sometest restore's acture for a regar